In [0]:
spark.conf.set(
  "fs.azure.account.key.goodreadsreviews60301813.dfs.core.windows.net",
  "sI48e0EntAgjv6jZU0Rhqmk2WWR0vsfHYc/urFlpLKbOusdGyHZgpJjOG8ek5VCCktAYpYjh49PC+AStvm6tbg=="
)

dbutils.fs.ls("abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/")

train_df = spark.read.format("delta").load(
    "abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/feature_v2/train/"
)


%restart_python


In [0]:
# ============================================================
# 📘 Goodreads Text Feature Engineering - Step III.2–III.3
# ============================================================

import re
import emoji
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

# 1️⃣ Load the curated Gold training dataset (feature_v2/train)
train_df = spark.read.format("delta").load(
    "abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/feature_v2/train/"
)

print(f"✅ Loaded training dataset with {train_df.count():,} rows")
train_df.select("review_text").show(5, truncate=False)

# ============================================================
# 2️⃣ Define text-cleaning UDF
# ============================================================

def clean_text(text):
    if text is None:
        return None

    # Convert to lowercase
    text = text.lower()

    # Replace URLs with <URL>
    text = re.sub(r"http\S+|www\S+|https\S+", "<URL>", text)

    # Replace numbers with <NUM>
    text = re.sub(r"\d+", "<NUM>", text)

    # Replace emojis with <EMOJI>
    text = emoji.replace_emoji(text, replace="<EMOJI>")

    # Remove punctuation (keep basic sentence marks if you want)
    text = re.sub(r"[^\w\s<>]", "", text)

    # Remove extra spaces
    text = re.sub(r"\s+", " ", text)

    # Trim spaces
    text = text.strip()

    return text

# Register the UDF so Spark can apply it in parallel
clean_text_udf = F.udf(clean_text, StringType())

# ============================================================
# 3️⃣ Apply cleaning to review_text column
# ============================================================

cleaned_df = (
    train_df
    .withColumn("clean_text", clean_text_udf(F.col("review_text")))
    # Filter out empty or very short reviews (<10 characters)
    .filter(F.length(F.col("clean_text")) >= 10)
)

# ============================================================
# 4️⃣ Verify results
# ============================================================

print("✅ Cleaned text sample:")
cleaned_df.select("review_text", "clean_text").show(10, truncate=False)
print(f"✅ Remaining rows after filtering: {cleaned_df.count():,}")

# ============================================================
# 5️⃣ (Optional) Save cleaned dataset to Gold feature_v2/clean_train
# ============================================================

cleaned_df.write.format("delta") \
    .mode("overwrite") \
    .save("abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/feature_v2/clean_train/")

print("✅ Cleaned text saved to gold/feature_v2/clean_train/")


✅ Loaded training dataset with 5,140 rows
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|review_text                                                                           

In [0]:
# Install only once (skip if already done)
%pip install nltk textblob

import nltk
nltk.download('vader_lexicon')

from nltk.sentiment import SentimentIntensityAnalyzer
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/40.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━ 30.7/40.5 kB 837.2 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━ 30.7/40.5 kB 837.2 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 342.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━ 51.2/57.7 kB 1.8 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.5 MB ? eta -:--:--
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.5 MB 2.7 MB/s eta 0:00:01
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.5 MB 2.5 MB/s eta 0:00:01
   ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.5 MB 1.4 MB/s eta 0:00:01
   ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.5 MB 2.2 MB/s eta 0:00:01
   ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.5 MB 2.3 MB/s eta 0:00:01
   ━

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/spark-3883e911-b45b-496b-8a7d-50/nltk_data...


In [0]:
cleaned_df = spark.read.format("delta").load(
    "abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/feature_v2/train_cleaned/"
)


In [0]:
df_features = cleaned_df.withColumn(
    "review_length_words", F.size(F.split(F.col("review_text"), " "))
).withColumn(
    "review_length_chars", F.length(F.col("review_text"))
)


In [0]:
from pyspark.sql.types import StructType, StructField, FloatType

sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    if text is None or text.strip() == "":
        return {"neg": 0.0, "neu": 0.0, "pos": 0.0, "compound": 0.0}
    scores = sia.polarity_scores(text)
    return {
        "neg": float(scores["neg"]),
        "neu": float(scores["neu"]),
        "pos": float(scores["pos"]),
        "compound": float(scores["compound"]),
    }

# Define the schema of the returned dictionary
sentiment_schema = StructType([
    StructField("neg", FloatType()),
    StructField("neu", FloatType()),
    StructField("pos", FloatType()),
    StructField("compound", FloatType())
])

# Register UDF
get_sentiment_udf = F.udf(get_sentiment, sentiment_schema)

# Apply UDF
df_features = df_features.withColumn("sentiment", get_sentiment_udf(F.col("review_text")))


In [0]:
df_features = df_features \
    .withColumn("sentiment_neg", F.col("sentiment.neg")) \
    .withColumn("sentiment_neu", F.col("sentiment.neu")) \
    .withColumn("sentiment_pos", F.col("sentiment.pos")) \
    .withColumn("sentiment_compound", F.col("sentiment.compound")) \
    .drop("sentiment")


In [0]:
cleaned_df = spark.read.format("delta").load(
    "abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/feature_v2/train/"
)

df_features = cleaned_df \
    .withColumn("review_length_words", F.size(F.split(F.col("review_text"), " "))) \
    .withColumn("review_length_chars", F.length(F.col("review_text")))



In [0]:
from pyspark.sql import functions as F

# ✅ Step 1: Load the correct Gold dataset
cleaned_df = spark.read.format("delta").load(
    "abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/feature_v2/train/"
)

# ✅ Step 2: Add basic and sentiment features
df_features = cleaned_df \
    .withColumn("review_length_words", F.size(F.split(F.col("review_text"), " "))) \
    .withColumn("review_length_chars", F.length(F.col("review_text"))) \
    .withColumn("sentiment_pos", F.lit(None).cast("double")) \
    .withColumn("sentiment_neg", F.lit(None).cast("double")) \
    .withColumn("sentiment_compound", F.lit(None).cast("double"))  # placeholder if not computed yet

# ✅ Step 3: Preview
display(df_features.select(
    "review_text", 
    "review_length_words", 
    "review_length_chars", 
    "sentiment_pos", 
    "sentiment_neg", 
    "sentiment_compound"
).limit(5))


review_text,review_length_words,review_length_chars,sentiment_pos,sentiment_neg,sentiment_compound
"Quick, easy, fun read. A story of childhood friends, addie and Val who grow apart in high school and reunite later in life. A bit predictable with a happy ending.",30,162,null,null,null
"This is probably more like 3.5 stars. It's a young adult, Newberry book, so it's perhaps geared more for that audience in style, but I still found it interesting. I like books with quirky characters, and that's what this has. It's also a great story of discrimination in our own country towards Italian-Americans. We often don't think of discrimination against this group, so it was enlightening. I think adults who like young adult books will enjoy it, and young adults (in my opinion) should add it to their ""should read"" lists.",91,530,null,null,null
"-Slow and rambling. Terrifying in its statements of wars of extermination and the force of propaganda. -Excellent for understanding the psychology and neuroses of future dictators and megalomaniacs. For example: Hitler's interest in his 'method' of reading - finding only the most relevant bits of information - could be interpreted as a demand for facts which fit pre-determined worldview. Obsessive patriotism as refuge from conflicted family life. Failed artist leading to view as romantic hero/savior/Lord of German Volk - note interest in Wagner. -Not a 'good' book by any means, but one that must be read.",98,615,null,null,null
"I was given a copy of this book by Netgalley for an honest review. Worth the Wait: A Sexy Summer Read (The Guthrie Brothers) is one to not miss out on! Nathan works part-time at Violet Shaw's restaurant as the ""Barbecue master"", he also works as the Sheriff. It is hard for Nathan to trust women, since his dead wife had an affair. He will now have to learn how to open his heart and trust once again. Violet has no time for a relationship while running the restaurant, but can she make time for Nathan? This is such a great story, heartwarming and romantic. Nathans son is perfect also. He will be a heart breaker! I have loved getting to know these characters and falling in love with them also! I always enjoy a Lori Foster novel. I know that I will not be disappointed. If you have not read this series, I recommend that you do. I give WORTH THE WAIT 5/5 STARS.",166,867,null,null,null
A vampire who faints at the sight of blood? Hilarious!,10,54,null,null,null


In [0]:
# --- TF-IDF Feature Extraction ---
from pyspark.sql import functions as F
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# ⚙️ Convert Spark DF to Pandas for sklearn
pandas_df = df_features.select("review_id", "review_text").toPandas()

# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(
    max_features=5000,          # limit vocabulary
    stop_words='english',       # remove common words
    ngram_range=(1, 2)          # unigrams + bigrams
)

# Fit & transform
tfidf_matrix = tfidf.fit_transform(pandas_df["review_text"].fillna(""))

# Convert to DataFrame
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=[col.replace(' ', '_').replace(';', '_').replace('{', '_').replace('}', '_').replace('(', '_').replace(')', '_').replace('\n', '_').replace('\t', '_').replace('=', '_') for col in tfidf.get_feature_names_out()]
)
tfidf_df["review_id"] = pandas_df["review_id"]

# Merge back to Spark DF (optional)
tfidf_spark = spark.createDataFrame(tfidf_df)

# Save TF-IDF to Gold layer
tfidf_spark.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/feature_v2/tfidf/"
)
print("✅ TF-IDF features saved to Gold Zone.")


✅ TF-IDF features saved to Gold Zone.


In [0]:
# --- Semantic Embedding with Sentence-BERT ---
%pip install sentence-transformers

from sentence_transformers import SentenceTransformer
import numpy as np

# Load model (small + fast for Databricks)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Encode text to 384-dimensional embeddings
embeddings = model.encode(pandas_df["review_text"].fillna("").tolist(), show_progress_bar=True)

# Convert to DataFrame
embeddings_df = pd.DataFrame(embeddings)
embeddings_df["review_id"] = pandas_df["review_id"]

# Merge back and save
embeddings_spark = spark.createDataFrame(embeddings_df)
embeddings_spark.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/feature_v2/embeddings/"
)
print("✅ BERT embeddings saved to Gold Zone.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/44.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━ 30.7/44.0 kB 794.9 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━ 30.7/44.0 kB 794.9 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 384.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/488.0 kB ? eta -:--:--
   ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/488.0 kB 2.1 MB/s eta 0:00:01
   ━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/488.0 kB 1.9 MB/s eta 0:00:01
   ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/488.0 kB 1.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 256.0/488.0 kB 1.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━ 276.5/488.0 kB 1.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━ 460.8/488.0 kB 2.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/899.7 MB

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/161 [00:00<?, ?it/s]

✅ BERT embeddings saved to Gold Zone.


In [0]:
train_df = spark.read.format("delta").load(
    "abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/feature_v2/train/"
)
tfidf_df = spark.read.format("delta").load(
    "abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/feature_v2/tfidf/"
)
embeddings_df = spark.read.format("delta").load(
    "abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/feature_v2/embeddings/"
)


In [0]:
# Combine base, TF-IDF, and embedding features
combined_df = (
    train_df
    .join(tfidf_df, on="review_id", how="left")
    .join(embeddings_df, on="review_id", how="left")
)


In [0]:
# 🧩 Ensure all column names are unique before saving
from collections import Counter

cols = combined_df.columns
counts = Counter(cols)

new_cols = []
seen = {}
for c in cols:
    if counts[c] > 1:
        seen[c] = seen.get(c, 0) + 1
        new_cols.append(f"{c}_{seen[c]}")
    else:
        new_cols.append(c)

combined_df = combined_df.toDF(*new_cols)

# 💾 Save safely to Gold zone
combined_df.coalesce(10).write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("abfss://lakehouse60301813@goodreadsreviews60301813.dfs.core.windows.net/gold/features_v2/")

print("✅ Final combined feature dataset successfully saved — all duplicate column names fixed.")


✅ Final combined feature dataset successfully saved — all duplicate column names fixed.


In [0]:
# 👀 Inspect column names after deduplication
print("✅ Combined DataFrame loaded successfully.")
print("Total number of columns:", len(combined_df.columns))

# Show first 50 column names to check what 'rating' was renamed to
print("\n🔍 Sample of column names (first 50):")
for i, c in enumerate(combined_df.columns[:50], start=1):
    print(f"{i}. {c}")

# If you want to search for a specific column like 'rating'
print("\n🧩 Columns that contain the word 'rating':")
print([c for c in combined_df.columns if 'rating' in c.lower()])


✅ Combined DataFrame loaded successfully.
Total number of columns: 5392

🔍 Sample of column names (first 50):
1. review_id
2. book_id
3. title_1
4. author_id
5. author_name
6. user_id
7. rating_1
8. review_text
9. 000
10. 10_1
11. 100_1
12. 11_1
13. 12_1
14. 13_1
15. 14_1
16. 15_1
17. 16_1
18. 17_1
19. 18_1
20. 19_1
21. 1st
22. 20_1
23. 200_1
24. 2011
25. 2012
26. 2013
27. 2014
28. 2015
29. 2016
30. 2017
31. 20th
32. 20th_century
33. 21st
34. 21st_century
35. 24_1
36. 25_1
37. 2nd
38. 30_1
39. 3rd
40. 40_1
41. 400
42. 50_1
43. 60_1
44. 75_1
45. 80_1
46. 90_1
47. 90_minutes
48. 99_1
49. abandoned
50. abby

🧩 Columns that contain the word 'rating':
['rating_1', 'frustrating', 'rating_2', 'rating_stars', 'ratings', 'star_rating']
